In [2]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [4]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [5]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [6]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=20,
    alpha=0.2,
)

In [7]:
question = "Một người 22 tuổi muốn lái xe ô tô chở người 25 chỗ. Người này:"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : tuổi tối_đa của người lái_xe ô_tô chở người ( kể_cả xe_buýt ) trên 29 chỗ ( không kể chỗ của người lái_xe ) , xe ô_tô chở người giường_nằm là đủ 57 tuổi đối_với nam , đủ 55 tuổi đối_với nữ .

Metadata: {'category': 'General', 'fine_amount': 'None', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: General\nMức phạt: None\nNội dung vi phạm: Tuổi tối đa của người lái xe ô tô chở người (kể cả xe buýt) trên 29 chỗ (không kể chỗ của người lái xe), xe ô tô chở người giường nằm là đủ 57 tuổi đối với nam, đủ 55 tuổi đối với nữ.'}

Score: 0.8340027928352356

Result 2:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : bằng lái_xe hạng d : cấp cho người lái_xe ô_tô chở người trên 29 chỗ , xe ô_tô chở người giường_nằm , ô_tô kéo rơ_moóc dưới 750 kg .

Metadata: {'category': 'General', 'fine_amount': 'None', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: General\nMức phạt: No